In [2]:
#Bunlar dersteki Python kütüphaneleri
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay, classification_report, accuracy_score
from sklearn.ensemble import VotingClassifier

file_path = 'data/diabetic+retinopathy+debrecen/messidor_features.arff' #arff dosyası bizim veriyi aldığımız dosyadır. Onu bir değşkene (obje) eşitliyoruz ki istediğimizde çağırabilelim.
data, meta = arff.loadarff(file_path)
df = pd.DataFrame(data)
print(df)


X = df.iloc[:, :-1] #Bu kod tüm satırları alırken, en son sütunu almıyor. Sebebi ise sadece y ekseninde bize lazım olması.

y = df.iloc[:, -1].to_frame() #Bu kod aynı şekilde tüm satırları alırken bu sefer sadece son sütunu alıyor.

# y'deki byte stringleri (b'0', b'1') temiz sayıya (0, 1) çevirme
# (Bu, veriyi modelin anlayacağı sayısal formata getirir. Ders slaytlarında bu decoding vardı.)
y['Class'] = y['Class'].str.decode('utf-8').astype(int)

FileNotFoundError: [Errno 2] No such file or directory: 'data/diabetic+retinopathy+debrecen/messidor_features.arff'

In [ ]:
X.info() #Bu kod satırı bize verilerde herhangi bir null değer var mı onu gösteriyor. 0'dan 18'e kadar olan sütunların hepsindeki 1151 satırdaki değerler non-null. O yüzden eksik veri yok. Yani imputation işlemi yapmıyoruz.
X.head()

In [ ]:
y.value_counts() #Bu kod satırı da bize verilerimizin degeli olduğunu gösteriyor. Yani hasta olanlar ve olmayanların sayısı orantısal olarak birbirine yakın olduğundan veri setimizi ekstradan düzenlememize gerek yok.

In [ ]:
scaler = StandardScaler()#Scaler tanımlıyoruz. Değeri 1000'li olanla 10'lu olan aynı ağırlıkta. KNN'de falan önemli bunlar.

X_train_scaled = scaler.fit_transform(X_train)#Standadizasyon için standart sapma lazım. Onu sadece burda hesaplıyoruz. Sonra X_traini standardize ediyoruz.

X_test_scaled = scaler.transform(X_test)#Standardizasyonu burda da yapıyoruz ama burda standart sapmayı bulmuyoruz. Burda standart sapma bulma yok çünkü testten bu değeri alırsak model buna göre eğitilebilir. Overfit olabilir.

In [ ]:
#Burdan sonra birkaç model eğitip, iyi olanı seçmeye çalışacağız. F1 falan filan burda devreye giriyor.
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(random_state=42, gamma="auto"),
    "Random Forest": RandomForestClassifier(random_state=42),
}
results = {}

for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train.values.ravel(), scoring = "f1", cv=5)#Cross validation 5 ile F1 hesabı.
    results[name] = cv_scores.mean()

print(pd.Series(results).sort_values(ascending=False))
#Neden F1 bu kadar önemli. Çünkü Precision veya Recall'a ayrı ayrı bakmaktansa, ikisininde belli miktar yüksek olması gerekiyor. Yani hem yanlış alarm vermemesini, hem de halihazırda hasta olanları kaçırmamak amacımız.

In [ ]:
parameters = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]}, {'kernel': ['rbf'], 'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1]}#Burdaki değerleri slaytlardan ve internetten aldık. Genel olarak bu değerler kullanılıyor.
]
svm = SVC(random_state=42)
grid_search_svm = GridSearchCV(svm, parameters, cv=5, scoring='f1', verbose=2, n_jobs=-1)
grid_search_svm.fit(X_train_scaled, y_train.values.ravel())#Grid Search çalıştır. Bu bize en iyi ayarı buluyor. Çalışma mantığı o yazdıklarımızın kombinasyonlarını deniyor.

print(f"Parametre sonuçları: {grid_search_svm.best_params_}")
print(f"En iyi parametrelerle F1 Skoru: {grid_search_svm.best_score_:.4f}")

In [ ]:
#Değerleri yine slaytlardan ve internetten bulduk.
parameters = [
    {'C': [0.1, 1, 10, 100], 'solver': ['lbfgs', 'liblinear']}
]

logistic = LogisticRegression(max_iter=5000, random_state=42)

#Yine Grid Search yapıyoruz. Ve yine cross-val 5'te.
grid_search_lr = GridSearchCV(logistic, parameters, cv=5, scoring='f1')

grid_search_lr.fit(X_train_scaled, y_train.values.ravel())

print(f"Parametre sonuçları: {grid_search_lr.best_params_}")
print(f"En iyi parametrelerle F1 Skoru: {grid_search_lr.best_score_:.4f}")

In [ ]:
#Random Forest modelini eğitiyoruz. Parametreler internetten slaytlardan vs.
parameters = [ {'n_estimators': [100, 200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [None, 10, 20], 'random_state': [42]}]

rf = RandomForestClassifier()

#Yukarıdakilerin aynısı. Grid Search kur sonra cross-val yap.
grid_search_rf = GridSearchCV(rf, parameters, cv=5, scoring='f1', verbose=2, n_jobs=-1)

grid_search_rf.fit(X_train_scaled, y_train.values.ravel())

print(f"Parametre sonuçları: {grid_search_rf.best_params_}")
print(f"En iyi parametrelerle F1 skoru: {grid_search_rf.best_score_:.4f}")

In [ ]:
#Normalde Ensemble Learning yoktu. Ancak CH7 indelendikten sonra koymaya karar verdik.
#Ayrıca KNN ile devam etmeme kararı aldık. Sebebi ise F1'inin düşük olması.
#Best Estimator ise en iyi halini alıyor modellerin. Parametreler bağlamında.
best_lr = grid_search_lr.best_estimator_
best_svm = grid_search_svm.best_estimator_
best_svm.set_params(probability=True)
best_rf = grid_search_rf.best_estimator_

# Burada soft voting seçtik. Sebebi ise sağlıkla alakalı bir model olduğundan çoğunluk oyuna bakmaktansa o oylar ne kadar veya modeller ne kadar emin kesin onu sorgulamak.
vc = VotingClassifier(estimators=[('lr', best_lr), ('svc', best_svm), ('rf', best_rf)], voting='soft')

# Cross-validation 5 ile F1'lere bakıyoruz
print(f"Ortalama F1 Skoru: {cross_val_score(vc, X_train_scaled, y_train.values.ravel(), cv=5, scoring='f1').mean():.4f}")

In [ ]:
# Burda Voting Classifier tüm eğitim verisiyle tekrar eğitiyoruz
# Cross validation sadece kontrol içindi, şimdi her şeyle öğreniyor.
vc.fit(X_train_scaled, y_train.values.ravel())

# Hiç görülmemiş test seti üzerinde tahmin yapıyoruz
y_pred = vc.predict(X_test_scaled)

print(classification_report(y_test, y_pred))

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy %{acc*100:.2f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Sağlıklılar (0)', 'Hastalar (1)'])
fig, ax = plt.subplots(figsize=(8, 6))
display.plot(cmap='Blues', ax=ax)
plt.title('Test Seti Verileri için Confusion Matrix')
plt.show()

In [ ]:
# Modelimizin olasılık tahminlerini alıyoruz. Sadece hasta yani 1 olma olasılığı.
y_probs = vc.predict_proba(X_test_scaled)[:, 1]

# ROC eğrisi değerleri
fpr, tpr, thresholds = roc_curve(y_test, y_probs)

# AUC skorunu
auc_score = roc_auc_score(y_test, y_probs)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Köşegen (şans çizgisi)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive')
plt.ylabel('True Positive')
plt.title('Final Model ROC Eğrisi')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()
print(f"AUC skoru: {auc_score:.4f}")